In [9]:
import sys
print(sys.executable)

c:\Users\91818\AppData\Local\Programs\Python\Python310\python.exe


In [10]:
import numpy as np
import pandas as pd
import os
import joblib
from sklearn.preprocessing import MinMaxScaler

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense

In [12]:
import yfinance as yf

In [13]:
available_companies = {
    'TCS': 'TCS.NS',
    'Infosys': 'INFY.NS',
    'Reliance': 'RELIANCE.NS',
    'HDFC Bank': 'HDFCBANK.NS',
    'ICICI Bank': 'ICICIBANK.NS',
    'Wipro': 'WIPRO.NS',
    'HCL Tech': 'HCLTECH.NS',
    'SBI': 'SBIN.NS',
    'Bharti Airtel': 'BHARTIARTL.NS',
    'L&T': 'LT.NS',
    'Bajaj Finance': 'BAJFINANCE.NS',
    'Axis Bank': 'AXISBANK.NS',
    'Maruti Suzuki': 'MARUTI.NS',
    'Asian Paints': 'ASIANPAINT.NS',
    'Titan': 'TITAN.NS',
    'Ultratech Cement': 'ULTRACEMCO.NS',
    'Nestle India': 'NESTLEIND.NS',
    'Tech Mahindra': 'TECHM.NS',
    'Power Grid': 'POWERGRID.NS',
    'Coal India': 'COALINDIA.NS',
    'ONGC': 'ONGC.NS',
    'JSW Steel': 'JSWSTEEL.NS',
    'NTPC': 'NTPC.NS',
    'Hindalco': 'HINDALCO.NS',
    'Sun Pharma': 'SUNPHARMA.NS'
}

In [ ]:
def get_stock_data(ticker, start='2015-01-01', end='2024-12-31'):
    df = yf.download(ticker, start=start, end=end)
    return df['Close'].values.reshape(-1, 1), df.index

def create_dataset(data, time_step=60):
    X, Y = [], []
    for i in range(len(data)-time_step-1):
        X.append(data[i:(i+time_step), 0])
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

In [15]:
def train_lstm(X_train, y_train):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    return model

def train_gru(X_train, y_train):
    model = Sequential()
    model.add(GRU(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(GRU(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    return model

In [16]:
def train_and_save_models():
    for company_name, ticker in available_companies.items():
        try:
            data, _ = get_stock_data(ticker)
            scaler = MinMaxScaler()
            data_scaled = scaler.fit_transform(data)
            X, y = create_dataset(data_scaled)
            X = X.reshape(X.shape[0], X.shape[1], 1)
            X_train, y_train = X[:int(0.8*len(X))], y[:int(0.8*len(y))]
            lstm_model = train_lstm(X_train, y_train)
            gru_model = train_gru(X_train, y_train)
            os.makedirs(f"models/{company_name}", exist_ok=True)
            lstm_model.save(f"models/{company_name}/lstm_model.h5")
            gru_model.save(f"models/{company_name}/gru_model.h5")
            joblib.dump(scaler, f"models/{company_name}/scaler.pkl")
        except Exception as e:
            print(f"Error training {company_name}: {e}")

train_and_save_models()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
c:\Users\91818\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
c:\Users\91818\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
c:\Users\91818\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_